In [35]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper


In [36]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)

wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)

In [37]:
# Custom tools [RAG Tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [38]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50).split_documents(docs)

embeddings = HuggingFaceEmbeddings()
vectorstore = FAISS.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever()


/opt/anaconda3/envs/venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [39]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool=create_retriever_tool(retriever,"langsmith-search","Search any information about Langsmith ")
retriever_tool.name 

'langsmith-search'

In [40]:
tools=[wiki,arxiv,retriever_tool]

In [42]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [44]:
from langchain_groq import ChatGroq
llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")

In [46]:
# Prompt template
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")

prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [47]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

In [50]:
# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(llm, tools, prompt)

In [51]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [52]:
agent_executor.invoke({"input":"tell me about langsmith"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith​PythonTypeScriptpip install -U langsmithyarn add langsmith2. Create an API key​To create an API key head to the Settings page. Then click Create API Key.3. Set up your environment​Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. Log your first trace​Tracing to LangSmith for LangChain u

{'input': 'tell me about langsmith',
 'output': 'Based on the output of the tool call, I can see that LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It allows developers to closely monitor and evaluate their applications, enabling them to ship quickly and with confidence. LangSmith provides a simple and straightforward way to create and manage API keys, set up environments, and log traces.\n\nFrom the provided information, I can gather that LangSmith is not directly related to LangChain, as it is mentioned that LangChain is not necessary to use LangSmith. LangSmith appears to be a standalone platform that can be used independently.\n\nI can also see that LangSmith provides a range of features, including tracing, evaluation, and metadata management. The platform seems to be designed to help developers build and manage LLM applications with ease.\n\nIn summary, LangSmith is a platform for building and managing production-grade LLM applicati

In [53]:
agent_executor.invoke({"input":"what is machinelearning"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `wikipedia` with `{'query': 'Machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks witBased on the tool call result, I can provide a direct response.

Machine learning is a field of study in artificial intelligence that enables computers to learn from data and make predictions or decisions without being explicitly programmed. It is a subset of artificial intelligence that involves the use of algorithms and statistical models to analyze and learn from large datasets. Machine learning is used in a wide range of applications, including image and speech recognition, natural language processing, and predictive analytics.

> Finished chain.


{'input': 'what is machinelearning',
 'output': 'Based on the tool call result, I can provide a direct response.\n\nMachine learning is a field of study in artificial intelligence that enables computers to learn from data and make predictions or decisions without being explicitly programmed. It is a subset of artificial intelligence that involves the use of algorithms and statistical models to analyze and learn from large datasets. Machine learning is used in a wide range of applications, including image and speech recognition, natural language processing, and predictive analytics.'}

In [55]:
agent_executor.invoke({"input":"What's the paper 1706.03762 about?"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on c
Invoking: `langsmith-search` with `{'query': 'Attention Is All You Need'}`


}],        model: "gpt-3.5-turbo",    });    return result.choices[0].message.content;});await pipeline("Hello, world!")// Out: Hello there! How can I assist you today?View a sample output trace.Learn more about tracing in the how-to guides.5. Run your first evaluation​Evaluation requires a system to test, data to serve as test cases, and optionally evaluators to grade the results. Here we use a built-in accuracy evaluator.PythonTypeScriptfrom langsmith import Clientfrom langsmith.evaluation import evaluateclient = Client()# Define dataset: these are your test casesdataset_name = "Sample Dataset"dataset = client.

{'input': "What's the paper 1706.03762 about?",
 'output': 'The paper "Attention Is All You Need" is a groundbreaking work in the field of natural language processing. It introduced the Transformer model, a novel architecture that revolutionized the way we approach sequence-to-sequence tasks. The Transformer model relies solely on self-attention mechanisms, eliminating the need for recurrent neural networks (RNNs) and convolutional neural networks (CNNs). This innovation has far-reaching implications for many applications, including machine translation, text summarization, and question answering.'}